# vessel-cable-anomaly-hunter
DTU Deep Learning project 29, group 80

## Required Libraries Installation
Run this in your terminal before executing this:

In [ ]:
pip install -r requirements.txt

## 1) Data Download

#### File imports for the data download and preparation

In [1]:
import config
import src.data.ais_downloader as ais_downloader
import src.data.ais_filtering as ais_filtering
import src.data.ais_reader as ais_reader
import src.data.ais_to_parquet as ais_to_parquet

import src.data.ais_reader as ais_reader
import src.pre_proc.ais_query as ais_query 
import src.utils.ais_maps as ais_maps

#### Library imports for the data download and preparation

In [2]:
from tqdm import tqdm
import gc
from pathlib import Path
import pandas as pd
from datetime import date, timedelta

#### Set data preferences and configuration inputs

In [3]:
VERBOSE_MODE = config.VERBOSE_MODE                          # Whether to print verbose output

START_DATE = config.START_DATE                              # Start date for data downloading
END_DATE   = config.END_DATE                                # End date for data downloading

AIS_DATA_NAME = config.AIS_DATA_FOLDER                      # Name of the folder to store AIS data
DELETE_DOWNLOADED_CSV = config.DELETE_DOWNLOADED_CSV        # Whether to delete raw downloaded CSV files after processing

VESSEL_AIS_CLASS = config.VESSEL_AIS_CLASS                  # AIS classes of vessels to include

REMOVE_ZERO_SOG_VESSELS = config.REMOVE_ZERO_SOG_VESSELS    # Whether to remove vessels with zero Speed Over Ground
SOG_IN_MS = config.SOG_IN_MS                                # If True, SOG is in meters/second; if False, SOG is in knots
SOG_MIN_KNOTS = config.SOG_MIN_KNOTS                        # Minimum SOG in knots
SOG_MAX_KNOTS = config.SOG_MAX_KNOTS                        # Maximum SOG in knots

BBOX = config.BBOX                                          # Bounding Box to prefilter AIS data
POLYGON_COORDINATES = config.POLYGON_COORDINATES            # Polygon coordinates for filter Area of Interest
COLUMNS_TO_DROP = config.COLUMNS_TO_DROP                    # Columns to drop from AIS data

#### Create Directories and Paths

In [4]:
# --- Create paths ---
folder_path = Path(AIS_DATA_NAME)
folder_path.mkdir(parents=True, exist_ok=True)
csv_folder_path = folder_path / "csv"
csv_folder_path.mkdir(parents=True, exist_ok=True)
parquet_folder_path = folder_path / "parquet"
parquet_folder_path.mkdir(parents=True, exist_ok=True)

file_port_locations = folder_path / "port_locodes.csv" # Path to port locations file

#### Main Script
1) Download one single .csv AIS data file from http://aisdata.ais.dk (link to data column description http://aisdata.ais.dk/!_README_information_CSV_files.txt);
2) For a given AOI in Denmark with known cable positions, filter AIS messages by cleansing unrealistic/unphysical messages or duplicates and removes error-prone messages within port areas;
3) Segmentation of vessel tracks per MMSI and date;
4) Parquet Conversion;

In [5]:
# --- If you want to download all csv files before, uncomment the line below ---
# ais_downloader.download_multiple_ais_data(START_DATE, END_DATE, folder_path)

# --- Build the schedule of download string dates ---
dates = ais_downloader.get_work_dates(START_DATE, END_DATE, csv_folder_path, filter=False)

# --- Iterate with tqdm and download, unzip and delete ---
for day in tqdm(dates, desc=f"Processing data", unit="file" ):
    tag = f"{day:%Y-%m}" if day < date.fromisoformat("2024-03-01") else f"{day:%Y-%m-%d}"
    print(f"\nProcessing date: {tag}")

    # --- Download one day ---
    csv_path = ais_downloader.download_one_ais_data(day, csv_folder_path)
    
    # --- Load CSV into DataFrame ---
    df_raw = ais_reader.read_single_ais_df(csv_path, BBOX, columns_to_drop=COLUMNS_TO_DROP, verbose=VERBOSE_MODE)
    # --- Optionally delete the downloaded CSV file ---
    if DELETE_DOWNLOADED_CSV: csv_path.unlink(missing_ok=True)
    
    # --- Filter and split ---
    # Filter AIS data, keeping Class A and Class B by default,
    df_filtered = ais_filtering.filter_ais_df(
        df_raw,                                               # raw AIS DataFrame
        polygon_coords=POLYGON_COORDINATES,                   # polygon coordinates for precise AOI filtering
        allowed_mobile_types=VESSEL_AIS_CLASS,                # vessel AIS class filter
        apply_polygon_filter=True,                            # keep polygon filtering enabled boolean
        remove_zero_sog_vessels=REMOVE_ZERO_SOG_VESSELS,      # use True/False to enable/disable 90% zero-SOG removal
        output_sog_in_ms=SOG_IN_MS,                           # convert SOG from knots in m/s (default) boolean
        sog_min_knots=SOG_MIN_KNOTS,                          # min SOG in knots to keep (None to disable)
        sog_max_knots=SOG_MAX_KNOTS,                          # max SOG in knots to keep (None to disable) 
        port_locodes_path=file_port_locations,                # path to port locodes CSV
        exclude_ports=True,                                   # exclude port areas boolean 
        verbose=VERBOSE_MODE,                                 # verbose mode boolean
    )
    
    # Free df_raw memory
    del df_raw
    gc.collect()

    # --- Parquet conversion ---
    # Save to Parquet by MMSI
    ais_to_parquet.save_by_mmsi(
        df_filtered,                                             # filtered AIS DataFrame 
        verbose=VERBOSE_MODE,                                    # verbose mode boolean
        output_folder=parquet_folder_path                        # output folder path
    )

    # Free df_filtered memory
    del df_filtered
    gc.collect()

Processing data:   0%|          | 0/30 [00:00<?, ?file/s]


Processing date: 2025-08-01
Skipping 2025-08-01 download: already present in ais-data/csv folder
Read AIS data: 1,128,873 rows within bbox, 511 unique vessels
 [filter_ais_df] Before filtering: 1,128,873 rows, 511 vessels
 [filter_ais_df] Type filtering: 1,093,101 rows (removed 35,772) using ['Class A', 'Class B']
 [filter_ais_df] MMSI filtering: 1,093,079 rows, 508 vessels
 [filter_ais_df] Duplicate removal: 638,467 rows, 508 vessels
 [filter_ais_df] Polygon filtering: 337,847 rows (removed 300,620), 378 vessels
 [filter_ais_df] Port-area removal: removed 160,778 rows in 3 overlapping ports
 [filter_ais_df] COG sanity: 175,598 rows (removed 1,471) with range [0, 360] deg
 [filter_ais_df] SOG sanity: 173,215 rows (removed 2,381) with range [0.5, 35.0] knots
 [filter_ais_df] Final: 173,215 rows, 327 unique vessels (SOG in m/s)
 [save_by_mmsi] Removed existing partitions for 327 (MMSI, Date) combinations.


Processing data:   3%|▎         | 1/30 [00:08<04:16,  8.86s/file]

 [save_by_mmsi] Parquet dataset written/appended at: /dtu/blackhole/0e/213550/dark-vessel-hunter/ais-data/parquet

Processing date: 2025-08-02
Skipping 2025-08-02 download: already present in ais-data/csv folder
Read AIS data: 1,161,214 rows within bbox, 452 unique vessels
 [filter_ais_df] Before filtering: 1,161,214 rows, 452 vessels
 [filter_ais_df] Type filtering: 1,120,127 rows (removed 41,087) using ['Class A', 'Class B']
 [filter_ais_df] MMSI filtering: 1,114,486 rows, 447 vessels
 [filter_ais_df] Duplicate removal: 616,865 rows, 447 vessels
 [filter_ais_df] Polygon filtering: 314,494 rows (removed 302,371), 342 vessels
 [filter_ais_df] Port-area removal: removed 168,854 rows in 3 overlapping ports
 [filter_ais_df] COG sanity: 145,303 rows (removed 337) with range [0, 360] deg
 [filter_ais_df] SOG sanity: 144,010 rows (removed 1,291) with range [0.5, 35.0] knots
 [filter_ais_df] Final: 144,010 rows, 290 unique vessels (SOG in m/s)
 [save_by_mmsi] Removed existing partitions for 2

Processing data:   7%|▋         | 2/30 [00:18<04:23,  9.43s/file]

...

Processing data:  97%|█████████▋| 29/30 [14:56<00:50, 50.38s/file]


Processing date: 2025-08-30
Starting download and extraction for 2025-08-30


Unzipping into ais-data/csv folder : 100%|██████████| 1/1 [00:18<00:00, 18.10s/it]


Deleted downloaded zip file for 2025-08-30
Completed download and extraction for 2025-08-30
Read AIS data: 1,220,338 rows within bbox, 346 unique vessels
 [filter_ais_df] Before filtering: 1,220,338 rows, 346 vessels
 [filter_ais_df] Type filtering: 1,175,090 rows (removed 45,248) using ['Class A', 'Class B']
 [filter_ais_df] MMSI filtering: 1,172,421 rows, 343 vessels
 [filter_ais_df] Duplicate removal: 555,626 rows, 343 vessels
 [filter_ais_df] Polygon filtering: 250,792 rows (removed 304,834), 228 vessels
 [filter_ais_df] Port-area removal: removed 131,407 rows in 3 overlapping ports
 [filter_ais_df] COG sanity: 118,470 rows (removed 915) with range [0, 360] deg
 [filter_ais_df] SOG sanity: 116,550 rows (removed 1,920) with range [0.5, 35.0] knots
 [filter_ais_df] Final: 116,550 rows, 195 unique vessels (SOG in m/s)
 [save_by_mmsi] Removed existing partitions for 195 (MMSI, Date) combinations.
 [save_by_mmsi] Parquet dataset written/appended at: /dtu/blackhole/0e/213550/dark-vessel-

Processing data: 100%|██████████| 30/30 [15:42<00:00, 31.41s/file]


## 2) Pre-processing

#### File imports for the pre-processing

In [1]:
import config
import src.pre_proc.pre_processing_utils as pre_processing_utils
import src.pre_proc.ais_query as ais_query
import src.pre_proc.ais_segment as ais_segment


#### Library imports for the pre-processing

In [2]:
from pathlib import Path
import pandas as pd
import json

#### Set pre-processing preferences and input configuration

In [3]:
VERBOSE_MODE = config.VERBOSE_MODE

FOLDER_NAME = config.AIS_DATA_FOLDER
folder_path = Path(FOLDER_NAME)
parquet_folder_path = folder_path / "parquet"

MAX_TIME_GAP_SEC = config.MAX_TIME_GAP_SEC
MAX_TRACK_DURATION_SEC = config.MAX_TRACK_DURATION_SEC
MIN_TRACK_DURATION_SEC = config.MIN_TRACK_DURATION_SEC
MIN_SEGMENT_LENGTH = config.MIN_SEGMENT_LENGTH

MIN_FREQ_POINTS_PER_MIN = config.MIN_FREQ_POINTS_PER_MIN
NUMERIC_COLS = config.NUMERIC_COLS

TRAIN_START_DATE = config.TRAIN_START_DATE
TRAIN_END_DATE = config.TRAIN_END_DATE

TEST_START_DATE = config.TEST_START_DATE
TEST_END_DATE = config.TEST_END_DATE

RESAMPLING_RULE = config.RESAMPLING_RULE

#### Pre processing function

In [4]:
def main_pre_processing(dataframe_type: str = "all"):

    if dataframe_type == "all":
        main_pre_processing("train")
        main_pre_processing("test")
        return
        
    elif dataframe_type == "train":
        print(f"[pre_processing] Querying AIS data for training period: {TRAIN_START_DATE} to {TRAIN_END_DATE}")
        # Loading filtered data from parquet files
        df = ais_query.query_ais_duckdb(parquet_folder_path, date_start=TRAIN_START_DATE, date_end=TRAIN_END_DATE, verbose=VERBOSE_MODE)
        
    elif dataframe_type == "test":
        print(f"[pre_processing] Querying AIS data for testing period: {TEST_START_DATE} to {TEST_END_DATE}")
        # Loading filtered data from parquet files
        df = ais_query.query_ais_duckdb(parquet_folder_path, date_start=TEST_START_DATE, date_end=TEST_END_DATE, verbose=VERBOSE_MODE)
    else:
        raise ValueError(f"Invalid dataframe_type: {dataframe_type}. Must be 'train' or 'test'.")
    
    # Converting COG to sine and cosine components
    df = pre_processing_utils.cog_to_sin_cos(df)
    
    # Dropping unnecessary columns and rows with missing values
    df.drop(columns=[ 
        'Type of mobile', 
        'COG', 
        'Date'], inplace=True, errors='ignore')
    
    # Removing rows with NaN values in essential columns
    df.dropna(inplace=True)
    
    # Grouping Ship types
    commercial_types = ["Cargo", "Tanker"]
    passenger_types = ["Passenger", "Pleasure", "Sailing"]
    service_types = ["Dredging", "Law enforcement", "Military", "Port tender", "SAR", "Towing", "Towing long/wide","Tug"]
    valid_types =  ["Fishing", "Service", "Commercial", "Passenger"]

    df.loc[df["Ship type"].isin(commercial_types), "Ship type"] = "Commercial"
    df.loc[df["Ship type"].isin(passenger_types), "Ship type"] = "Passenger"
    df.loc[df["Ship type"].isin(service_types), "Ship type"] = "Service"
    df.loc[~df["Ship type"].isin(valid_types), "Ship type"] = "Other"

    print("[pre_processing] Ship type counts:")
    print(df["Ship type"].value_counts())
    
    if VERBOSE_MODE:
        print(f"[pre_processing] DataFrame after dropping unnecessary columns and NaNs: {len(df):,} rows")

    # Segmenting AIS tracks based on time gaps and max duration, filtering short segments
    df = ais_segment.segment_ais_tracks(
        df,
        max_time_gap_sec=MAX_TIME_GAP_SEC,
        max_track_duration_sec=MAX_TRACK_DURATION_SEC,
        min_track_duration_sec=MIN_TRACK_DURATION_SEC,
        min_track_len=MIN_SEGMENT_LENGTH,
        verbose=VERBOSE_MODE
    )

    # Adding segment nr feature
    df = pre_processing_utils.add_segment_nr(df)

    # Removing segments with low point density
    df = pre_processing_utils.remove_notdense_segments(df, min_freq_points_per_min=MIN_FREQ_POINTS_PER_MIN)

    # Resampling all tracks to fixed time intervals
    df = pre_processing_utils.resample_all_tracks(df, rule=RESAMPLING_RULE)

    print(f"[pre_processing] Number of segments and rows after removing low-density segments and resampling: {df['Segment_nr'].nunique():,} segments, {len(df):,} rows")
    # Normalizing numeric columns
    df, mean, std = pre_processing_utils.normalize_df(df, NUMERIC_COLS)

    # Ship type labeling (mapping to be used later)
    df, ship_type_to_id = pre_processing_utils.label_ship_types(df)
    
    # Saving pre-processed DataFrame
    if dataframe_type == "train":
        print(f"[pre_processing] Saving pre-processed DataFrame to {config.PRE_PROCESSING_DF_TRAIN_PATH}...")
        output_path = config.PRE_PROCESSING_DF_TRAIN_PATH
        metadata_path = config.PRE_PROCESSING_METADATA_TRAIN_PATH
    else:
        print(f"[pre_processing] Saving pre-processed DataFrame to {config.PRE_PROCESSING_DF_TEST_PATH}...")
        output_path = config.PRE_PROCESSING_DF_TEST_PATH
        metadata_path = config.PRE_PROCESSING_METADATA_TEST_PATH

    if VERBOSE_MODE: print(f"[pre_processing] Columns of pre-processed DataFrame:\n{df.columns.tolist()}")
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(output_path, index=False)

    # Saving preprocessing metadata
    if VERBOSE_MODE: print(f"[pre_processing] Saving preprocessing metadata to {metadata_path}...")
    meta = {
        "mean": mean.tolist(),
        "std": std.tolist(),
        "ship_type_to_id": ship_type_to_id
    }

    with open(metadata_path, "w") as f:
        json.dump(meta, f, indent=4)

#### Pre processing script

In [5]:
main_pre_processing("train")

[pre_processing] Querying AIS data for training period: 2025-08-01 to 2025-08-28
[ais_query] Querying parquet files from: ais-data/parquet  from date 2025-08-01  to date  2025-08-28
[ais_query] 4,845,439 rows, 3,060 vessels, from date 2025-08-01 to date 2025-08-28
[pre_processing] Ship type counts:
Ship type
Commercial    1910924
Fishing       1681786
Passenger      962508
Service        157397
Other          132824
Name: count, dtype: int64
[pre_processing] DataFrame after dropping unnecessary columns and NaNs: 4,845,439 rows
[segment_ais_tracks] Starting with 4,845,439 rows, 3,060 unique vessels
[segment_ais_tracks] After segment-level filter: 4,838,589 rows, 7,331 segments
[pre_processing] Number of segments after removing low-density segments: 7,311
[pre_processing] Saving pre-processed DataFrame to ais-data/df_preprocessed/pre_processed_df_train.parquet...
[pre_processing] Columns of pre-processed DataFrame:
['Segment_nr', 'Timestamp', 'Latitude', 'Longitude', 'SOG', 'COG_sin', 'C

In [5]:
main_pre_processing("test")

[pre_processing] Querying AIS data for testing period: 2025-08-29 to 2025-08-30
[ais_query] Querying parquet files from: ais-data/parquet  from date 2025-08-29  to date  2025-08-30
[ais_query] 249,888 rows, 358 vessels, from date 2025-08-29 to date 2025-08-30
[pre_processing] Ship type counts:
Ship type
Commercial    140651
Passenger      45631
Fishing        44189
Service        13655
Other           5762
Name: count, dtype: int64
[pre_processing] DataFrame after dropping unnecessary columns and NaNs: 249,888 rows
[segment_ais_tracks] Starting with 249,888 rows, 358 unique vessels
[segment_ais_tracks] After segment-level filter: 249,208 rows, 427 segments
[pre_processing] Number of segments and rows after removing low-density segments and resampling: 411 segments, 43,224 rows
[pre_processing] Saving pre-processed DataFrame to ais-data/df_preprocessed/pre_processed_df_test.parquet...
[pre_processing] Columns of pre-processed DataFrame:
['Segment_nr', 'Timestamp', 'Latitude', 'Longitude

## Training

#### File imports for the training

#### Library imports for the training

#### Set training preferences

#### Script

## Evaluation

#### File imports for the evaluation

#### Library imports for the evaluation

#### Set evaluation preferences

#### Script

## Inspection